# Sandbox for testing code
## Project: Growing Urban Bicycle Networks

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Delete plots

In [ ]:
for folder, subfolders, files in os.walk(PATH["plots"]):
    for file in files:
        if not file.endswith('_analysis_poi_grid_betweenness.png'):
            os.remove(os.path.join(folder, file))
            
for folder, subfolders, files in os.walk(PATH["plots_networks"]):
    for file in files:
        if file.endswith('_analysis_poi_grid_betweenness.png'):
            os.remove(os.path.join(folder, file))

In [ ]:
for folder, subfolders, files in os.walk(PATH["plots"]):
    for file in files:
        if file.endswith('_noconstr.png'):
            os.remove(os.path.join(folder, file))

## Graph overlap

In [ ]:
%run -i functions.py
%run -i "../parameters/parameters.py"
for placeid, placeinfo in cities.items():

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    Gs = {}
    for networktype in networktypes:
        Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)   
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    
    # Test
    G1 = copy.deepcopy(Gs["biketrack"])
    G2 = copy.deepcopy(res["GTs"][-1])
    Ginter = intersect_igraphs(G1,G2)
    
    print(edge_lengths(G1), edge_lengths(G2), edge_lengths(Ginter))
    
    fig = plt.figure(figsize=(plotparam["bbox"][0]/plotparam["dpi"], plotparam["bbox"][1]/plotparam["dpi"]), dpi=plotparam["dpi"]); plt.axes().set_aspect('equal')
    map_center = nxdraw(G_carall, "carall")
    nxdraw(G1, "biketrack", map_center, list(set([v["id"] for v in G1.vs]).intersection(set([v["id"] for v in G_carall.vs]))))
    nxdraw(G2, "bikegrown_nonodes", map_center)
    nxdraw(Ginter, "highlight", map_center)
    

In [ ]:
Audio(sound_file, autoplay=True)

## Plot carminusbike network to see component growth (05_plot_results)

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting networks")
    
    # EXISTING INFRASTRUCTURE
    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    nodesize_poi = nodesize_from_pois(nnids)
    
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    if debug: pp.pprint(res)
        
    G_carminusbike = copy.deepcopy(G_carall)
    # PLOT all together (this can take some minutes)
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        delete_overlaps(G_carminusbike, GT)
        fig = initplot()
        nxdraw(G_carminusbike, "carall")
#         nxdraw(GT, "b'kegrown", map_center, nodesize = nodesize_grown)
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carminusGT_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()

## Order by city size

In [ ]:
!du '../data' | sort -h | awk '{print $2}' | cut -c9- | nl -w 2 -n ln | sed '$d' > '../cities_bysize.txt'

In [ ]:
with open('../cities_bysize.txt', 'r') as cities_bysize_in, open('../parameters/cities_all.csv', 'r') as cities_all_in, open('../parameters/cities_all_bysize.csv', 'w') as cities_all_bysize_out:
    cities_bysize_in = csv.reader(cities_bysize_in, delimiter='\t')
    cities_all_in = csv.reader(cities_all_in, delimiter=';')
    header = next(cities_all_in)
    cities_all_bysize_out = csv.writer(cities_all_bysize_out, delimiter=';')
    
    cities_bysize = {}
    
    
    cities_all_in_data = {}
    for row in cities_all_in:
        cities_all_in_data[row[0]] = [row[1], row[2]]
        
    rowstowrite = []
    for row in cities_bysize_in:
        rowstowrite.append([row[1], cities_all_in_data[row[1]][0], cities_all_in_data[row[1]][1]])
    
    cities_all_bysize_out.writerow(header) 
    cities_all_bysize_out.writerows(rowstowrite) 

## Move files for constricted calculations to folders

`find exports -iname "*railway*picklez" -exec cp '{}' export02/ \;`

## Move files into their folders

In [ ]:
# https://stackoverflow.com/questions/8858008/how-to-move-a-file

for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    for file in os.listdir(PATH["results"]):
        if file.startswith(placeid + "_"):
            os.rename(PATH["results"] + file, PATH["results"] + placeid + "/" + file)

## Test length issue

### csv_to_ox

In [ ]:
%run -i setup.py
%run -i functions.py
for placeid, placeinfo in cities.items():
    G_carall = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'carall')
    G_carall_WRONG = csv_to_ox_WRONG(PATH["data"] + placeid + "/", placeid, 'carall')

In [ ]:
G_carall_WRONG.edges(data=True)

In [ ]:
G_carall.edges(data=True)

### csv_to_ig, osm_to_ig

In [ ]:
%run -i setup.py
%run -i functions.py
for placeid, placeinfo in cities.items():
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')

In [ ]:
G_carall.es["weight"]

In [ ]:
for e in G_carall.es:
    if e["osmid"] == 27226011:
        print(e["weight"])

## Test shortest path between disconnected components

In [ ]:
G = ig.Graph(
        n=7, edges = [[0,1], [1,2], [0,4], [1,3], [4,3], [5,6]],
        edge_attrs={'weight': [1,1,1,1,1,1]}
    )

In [ ]:
indices = random.sample(list(G.vs), min(numnodepairs, len(G.vs)))

directness_links = np.zeros(int((len(indices)*(len(indices)-1))/2))
ind = 0
for c, v in enumerate(indices):
    poi_edges = G.get_shortest_paths(v, indices[c:], weights = "weight", output = "epath")
    print(v, poi_edges)
    for c_delta, path_e in enumerate(poi_edges[1:]): # Discard first empty list because it is the node to itself
        if path_e: # if path is non-empty = in same component
            distance_network = sum([G.es[e]['weight'] for e in path_e]) # sum over all edges of path
            distance_direct = 3.5 # dist first to last node, must be in format lat,lon = y, x

            directness_links[ind] = distance_direct / distance_network
            ind += 1
            
directness_links = directness_links[:ind]
directness_links